# Interact Demo with `interactive` and `InteractBase`

This notebook demonstrates how to use the `interactive` function and the `InteractBase` class from the `einteract` package to create interactive widgets in Jupyter Notebook.

We'll show two examples:
1. Using `interactive` to interact with multiple functions.
2. Using `InteractBase` to create a custom interactive widget.

**Note:** For better content representation, use same functionality under [ipyslides](https://github.com/asaboor-gh/ipyslides).

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as ipw
from einteract import interactive, ListWidget, plt2html

categories = ['A', 'B', 'C'] 
# Sample DataFrame
np.random.seed(0)
data = pd.DataFrame({
    'Category': np.random.choice(['A', 'B', 'C'], size=100),
    'Value': np.random.randn(100) * 10 + 50,
    'Score': np.random.randint(0, 100, size=100)
})

# Interactive Plotting
def plot_histogram(bins, category, column='Value'):
    plt.figure(figsize=(6,4))
    plt.hist(data[column], bins=bins, color='skyblue', edgecolor='black')
    plt.title(f'Histogram of {column}')
    plt.xlabel(column)
    plt.ylabel('Frequency')
    plt2html().display()


# Dynamic Data Filtering
def filter_data(bmax, category=0, min_score=0, max_score=100):
    filtered = data[
        (data['Category'] == categories[category or 0]) &
        (data['Score'] >= min_score) &
        (data['Score'] <= max_score)
    ]
    display(filtered.head())

# Custom Widget Layouts
def custom_ui(category, bins,max_score, min_score):
    subset = data[data['Category'] == categories[category or 0]]
    plt.figure(figsize=(3.5,4))
    plt.hist(subset['Value'], bins=bins, color='orange', edgecolor='black')
    plt.title(f'Category {categories[category or 0]} - Value Distribution')
    plt.xlabel('Value')
    plt.ylabel('Frequency')
    plt2html().display()

def set_bins_max(wbins,max_bins):
    wbins.max = max_bins
    print(f"Max bins: {max_bins}")

it = interactive(filter_data, custom_ui, plot_histogram, set_bins_max,
    category= ListWidget(options=[f'Category <em>{c}</em>' for c in categories], value=0, description='Select a Category'),
    min_score=ipw.IntSlider(min=0, max=100, value=0, description='Min Score:'),
    max_score=ipw.IntSlider(min=0, max=100, value=100, description='Max Score:'),
    wbins = ipw.fixed(ipw.IntSlider(min=5, max=30, value=10, description='Bins:')), # passed as widget
    bins = {'wbins': 'value'},
    max_bins = (10,100),
    bmax = {'wbins': 'max'}, # observe max value

    app_layout = {
        'left_sidebar': ['min_score', 'max_score', 'category', 'wbins','max_bins', 'out-0','out-3'], 
        'center': ['out-1','out-2'],
        'pane_widths': ['305px','1fr','0'], 'height': '500px'}
)
it

In [ ]:
it.set_css({
    'grid-gap': '4px',
    'padding': '4px',
    'background': '#eee',
    '.left-sidebar > *': {
        'background': '#ded'
    },
    '.center': {'grid-template-columns': '1fr 1fr',},
    '.center > *': {
        'background': '#ddd',
        'border-radius': '4px',
    }
})

# Same app as above through `InteractBase` class

In [ ]:
from einteract import InteractBase, callback
class MyInteract(InteractBase):
    def __init__(self, auto_update=True):
        super().__init__(auto_update= auto_update, # runtime decision
            grid_css = {
                'grid-gap': '4px',
                'padding': '4px',
                'background': '#eee',
                '.left-sidebar > *': {
                    'background': '#ded'
                },
                '.center': {'grid-template-columns': '1fr 1fr',},
                '.center > *': {
                    'background': '#ddd',
                    'border-radius': '4px',
                }
            },
            app_layout = {
                'left_sidebar': ['min_score', 'max_score', 'category', 'wbins','max_bins', 'out-0','out-3'], 
                'center': ['out-1','out-2'],
                'pane_widths': ['305px','1fr','0'], 'height': '500px',
            })
    def _interactive_params(self):
        return dict(
            category= ListWidget(options=[f'Category <em>{c}</em>' for c in categories], value=0, description='Select a Category'),
            min_score=ipw.IntSlider(min=0, max=100, value=0, description='Min Score:'),
            max_score=ipw.IntSlider(min=0, max=100, value=100, description='Max Score:'),
            wbins = ipw.fixed(ipw.IntSlider(min=5, max=30, value=10, description='Bins:')), # passed as widget
            bins = {'wbins': 'value'},
            max_bins = (10,100),
            bmax = {'wbins': 'max'}, # observe max value
        )

    @callback
    def filter_data(self, bmax, category=0, min_score=0, max_score=100):
        filtered = data[
            (data['Category'] == categories[category or 0]) &
            (data['Score'] >= min_score) &
            (data['Score'] <= max_score)
        ]
        display(filtered.head())

    @callback
    def custom_ui(self, category, bins,max_score, min_score):
        subset = data[data['Category'] == categories[category or 0]]
        plt.figure(figsize=(3.5,4))
        plt.hist(subset['Value'], bins=bins, color='orange', edgecolor='black')
        plt.title(f'Category {categories[category or 0]} - Value Distribution')
        plt.xlabel('Value')
        plt.ylabel('Frequency')
        plt2html().display()

    @callback
    def plot_histogram(self, bins, category, column='Value'):
        plt.figure(figsize=(6,4))
        plt.hist(data[column], bins=bins, color='skyblue', edgecolor='black')
        plt.title(f'Histogram of {column}')
        plt.xlabel(column)
        plt.ylabel('Frequency')
        plt2html().display()

    @callback
    def set_bins_max(self, wbins,max_bins):
        wbins.max = max_bins
        print(f"Max bins: {max_bins}")

    
MyInteract()